In [1]:
import os
import pyro
import torch
import pickle
import time

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import pyro.distributions as dist

from tqdm.auto import trange
from pyro.nn import PyroModule, PyroSample, PyroParam
from pyro.infer import MCMC, NUTS, SVI, Trace_ELBO, Predictive
from pyro.distributions import constraints
from pyro.infer.autoguide import AutoDiagonalNormal

os.chdir("../../")

from src.dgp_rff.deep_layer import DeepGP, DeepGPNoBias, DeepEnsembleGP,DeepGPFix
# from src.dgp_rff.get_variable import getvalue

In [2]:
from src.dgp_rff.get_variable import getvalue2

In [3]:
import torch
torch.cuda.is_available()
# trouble shoot see this link:https://stackoverflow.com/questions/77068908/how-to-install-pytorch-with-cuda-support-on-windows-11-cuda-12-no-matching

True

In [4]:
# start = time.perf_counter()
torch.cuda.current_device()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_device(device)
# cuda = torch.device('cuda') 

  # Next step

0. Get familiar with the coding structure
1. CPU -> GPU
2. Last step without bias
3. Figure out how they train
4. How to access the posterior mean and std/scale from the model
5. Learn pickle: numpy.ndarray, png
6. ....
7. construct a DeepGP class

In [5]:
# Set random seed for reproducibility
np.random.seed(1)

# Read data
cwd = os.getcwd()
print(cwd)

X_train_path = os.path.join(cwd, "folds", "synthetic_3_fold_1_X_train.txt")
X_test_path = os.path.join(cwd, "folds", "synthetic_3_fold_1_X_test.txt")
Y_train_path = os.path.join(cwd, "folds", "synthetic_3_fold_1_Y_train.txt")
Y_test_path = os.path.join(cwd, "folds", "synthetic_3_fold_1_Y_test.txt")

x_obs = np.loadtxt(X_train_path)
y_obs = np.loadtxt(Y_train_path)
x_val = np.loadtxt(X_test_path)
y_val = np.loadtxt(Y_test_path)

# Set plot limits and labels
xlims = [-0.2, 0.2]

# The X and Y have to be at least 2-dim
x_train = torch.from_numpy(x_obs).float()#.reshape(-1,1) 一般不需要reshape改变张量shape
y_train = torch.from_numpy(y_obs).float()
x_test = torch.from_numpy(x_val).float()#.reshape(-1,1)
y_test = torch.from_numpy(y_val).float()

C:\Users\yuanq\OneDrive\Desktopold\SB\research\DGPII\program\DGP-RFF-main


In [6]:
x_train = x_train.cuda()


In [7]:
y_train = y_train.cuda()
x_test = x_test.cuda()
y_test = y_test.cuda()

In [8]:
class ModelDGP(PyroModule):
    def __init__(self, dim_list=[1,1,1], J_list=[50,10]):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGP(dim_list, J_list)
        self.model.to('cuda')

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [9]:
class ModelDGPFix(PyroModule):
    def __init__(self, dim_list=[1,1,1], J_list=[50,10]):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGPFix(dim_list, J_list)
        self.model.to('cuda')

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [10]:
class ModelEnsembleDGP(PyroModule):
    def __init__(self, dim_list = [1,1,1], J_list=[50,10]):
        super().__init__()
        self.out_dim = dim_list[-1]
        self.model = DeepEnsembleGP(dim_list, J_list,2)
        self.model.to('cuda')
        
    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        # mu = self.linear(x1).squeeze()
        # batch shape | event shapt
        # 10000       |
        #scale = torch.FloatTensor([1e-8]*self.out_dim).cuda()
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample("obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)


            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [11]:
class ModelNoBiasDGP(PyroModule):
    def __init__(self, dim_list=[1,1,1], J_list=[50,10]):
        super().__init__()
        
        self.out_dim = dim_list[-1]
        self.model = DeepGPNoBias(dim_list, J_list)
        self.model.to('cuda')

    def forward(self, x, y=None):
        mu = self.model(x).squeeze() #10000*6
        
        # batch shape | event shapt
        # 10000       |
        
        scale = pyro.sample("sigma", dist.Gamma(torch.tensor(0.5, device='cuda'), torch.tensor(1.0, device='cuda'))).expand(self.out_dim)  # Infer the response noise

        # Sampling model
        with pyro.plate("data", x.shape[0]): # x.shape[0]=10000
            # obs = xxx("obs", mu, obs=y)
            obs = pyro.sample( "obs", dist.MultivariateNormal(mu.cuda(), torch.diag(scale * scale).cuda()), obs=y)
            
#         f1: phi(Omega x)W (+ epsilon1)
#         f2: phi(Omega f1)W (+ epsilon2)
        
#         f2 + epsilon ~ N(0, Sigma)
            
        return mu

In [12]:
print([x_train.shape[1],10,y_train.shape[1]])

[2, 10, 2]


In [13]:
#model = Model(in_dim_list=[x_train.shape[1],10], out_dim_list=[10,y_train.shape[1]], J_list=[50,10])
# mymodel_4layer = ModelDGP(dim_list=[x_train.shape[1],10,20,y_train.shape[1]], J_list=[50,10,20])
mymodel_4layer = ModelDGPFix(dim_list=[x_train.shape[1],10,10,y_train.shape[1]], J_list=[50,50,50])
print("x_train.shape",x_train.shape)
print(y_train.shape)
mymodel = mymodel_4layer.to('cuda')

[SingleGPFix(
  (layers): PyroModuleList(
    (0): SingleLayerFix(
      (layer): PyroLinear(in_features=200, out_features=10, bias=True)
    )
  )
), SingleGPFix(
  (layers): PyroModuleList(
    (0): SingleLayerFix(
      (layer): PyroLinear(in_features=200, out_features=10, bias=True)
    )
  )
), SingleGPFix(
  (layers): PyroModuleList(
    (0): SingleLayerFix(
      (layer): PyroLinear(in_features=200, out_features=2, bias=True)
    )
  )
)]
x_train.shape torch.Size([10000, 2])
torch.Size([10000, 2])


In [14]:
mymodel_3layer = ModelDGPFix(dim_list=[x_train.shape[1],10,y_train.shape[1]], J_list=[50,50])
print("x_train.shape",x_train.shape)
print(y_train.shape)
mymodel = mymodel_3layer.to('cuda')

[SingleGPFix(
  (layers): PyroModuleList(
    (0): SingleLayerFix(
      (layer): PyroLinear(in_features=200, out_features=10, bias=True)
    )
  )
), SingleGPFix(
  (layers): PyroModuleList(
    (0): SingleLayerFix(
      (layer): PyroLinear(in_features=200, out_features=2, bias=True)
    )
  )
)]
x_train.shape torch.Size([10000, 2])
torch.Size([10000, 2])


In [15]:
mean_field_guide = AutoDiagonalNormal(mymodel_4layer)
optimizer = pyro.optim.Adam({"lr": 0.001})

svi = SVI(mymodel_4layer, mean_field_guide, optimizer, loss=Trace_ELBO())
pyro.clear_param_store()

num_epochs = 15000
progress_bar = trange(num_epochs)
losslist = []
l = 500

interval = max(num_epochs//l, 1)
for epoch in progress_bar:
    loss = svi.step(x_train, y_train)
    if epoch % interval == 0:
        losslist.append(loss/ x_train.shape[0])
    progress_bar.set_postfix(loss=f"{loss / x_train.shape[0]:.3f}")

  0%|          | 0/15000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.plot(losslist)

In [ ]:
param_store = pyro.get_param_store()
for name, value in param_store.items():
    print(f"{name}: {value}")

In [ ]:
mymodel.parameters()

In [ ]:
predictive = Predictive(mymodel_4layer, guide=mean_field_guide, num_samples=500)
preds = predictive(x_test)

In [ ]:
y_pred = preds['obs'].cpu().detach().numpy().mean(axis=0)

In [ ]:
preds['obs'].shape

In [ ]:
y_obs.shape

In [ ]:
for d in range(y_train.shape[1]):
    plt.plot(x_obs, y_obs[:,d], label="Observation")
    plt.plot(x_test.cpu(), y_pred[:,d], label="Prediction")
    plt.legend()
    plt.show()

In [ ]:
getvalue2(mymodel_4layer,ensemble = False)

In [ ]:
dic_4 = getvalue2(mymodel_4layer,ensemble = False)

In [ ]:
dic_4_10000 = getvalue2(mymodel_4layer,ensemble = False)


In [ ]:
mean_field_guide = AutoDiagonalNormal(mymodel_3layer)
optimizer = pyro.optim.Adam({"lr": 0.001})

svi = SVI(mymodel_3layer, mean_field_guide, optimizer, loss=Trace_ELBO())
pyro.clear_param_store()

num_epochs = 15000
progress_bar = trange(num_epochs)
losslist = []
l = 500
interval = num_epochs//l
for epoch in progress_bar:
    loss = svi.step(x_train, y_train)
    if epoch % interval == 0:
        losslist.append(loss/ x_train.shape[0])
    progress_bar.set_postfix(loss=f"{loss / x_train.shape[0]:.3f}")

In [ ]:
plt.plot(losslist)   

In [ ]:
param_store = pyro.get_param_store()
for name, value in param_store.items():
    print(f"{name}: {value}")

In [ ]:
dic_3 = getvalue2(mymodel_4layer,ensemble = False)

In [ ]:
import numpy as np

def frobenius_distance(A, B):
    """
    计算两个矩阵之间的 Frobenius 范数距离。

    参数:
    A -- 第一个矩阵 (numpy 数组)
    B -- 第二个矩阵 (numpy 数组)

    返回值:
    Frobenius 范数距离 (浮点数)
    """
    # 检查两个矩阵的形状是否相同
    if A.shape != B.shape:
        return np.inf
        # raise ValueError("两个矩阵的形状必须相同")
    
    # 计算 Frobenius 范数距离
    distance = np.linalg.norm(A - B, 'fro')
    
    return distance


In [ ]:
A = dic_4_10000["0Weight"].cpu()
B = dic_3["0Weight"].cpu()
N = np.sqrt( np.linalg.norm(A) * np.linalg.norm(B))
d = frobenius_distance(A,B)/N
print(d)

In [ ]:
def cosine_similarity(A, B):
    a_flat = A.flatten()
    b_flat = B.flatten()
    if len(a_flat) != len(b_flat):
        return np.inf
    dot_product = np.dot(a_flat, b_flat)
    norm_a = np.linalg.norm(a_flat)
    norm_b = np.linalg.norm(b_flat)
    return dot_product / (norm_a * norm_b)

In [ ]:
#list_d = ["0Omega","0Weight","1Omega","1Weight"]
list_d = ["0Weight","1Weight"]
f_matrix = np.zeros((4, 4))
c_matrix = np.zeros((4, 4))
for i in range(2):
    key = list_d[i]
    A = dic_4[key].cpu()
    for j in range(2):
        key = list_d[j]
        B = dic_4_10000[key].cpu()
        N = np.sqrt( np.linalg.norm(A) * np.linalg.norm(B))
        d = frobenius_distance(A,B)/N
        c_matrix[i,j] = cosine_similarity(A,B)
        f_matrix[i,j]= d
    
print(c_matrix)

In [ ]:
print(f_matrix)

In [ ]:
def transfer_weights(mymodel, new_model):
    trained_state_dict = mymodel.state_dict()
    new_state_dict = new_model.state_dict()

    for name, param in trained_state_dict.items():
        if name in new_state_dict:
            try:
                new_state_dict[name].copy_(param)
                print(f"Copied {name} from trained model to new model.")
            except Exception as e:
                print(f"Could not copy parameter {name}: {e}")
        else:
            print(f"Parameter {name} not found in new model.")

    new_model.load_state_dict(new_state_dict)

transfer_weights(trained_model, new_model)

In [ ]:
trained_state_dict = mymodel.state_dict()
print(trained_state_dict)
for name, param in trained_state_dict.items():
    print(f"{name}: {param}")
    print("1")

In [ ]:
for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name).shape)

                  1 -> 50; 100 -> 6

Omega 1*50  1*50
W     6*100 6*100
bias  6     6
sigma 1     1